## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, Adam
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

num_classes = 10

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [28]:
learning_rate = 1e-3
epochs = 300

In [29]:
model = Sequential()
model.add(Conv2D(32,(3,3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

for i in range(2):
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
model.add(Conv2D(64,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [30]:
def cifar_generator(image_array, batch_size=128):
    while True:
        for indexs in range(0, len(image_array), batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            yield images, labels

In [31]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(factor=0.5, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=5, 
                              verbose=1)

In [32]:
model.fit(cifar_generator(x_train),
          epochs=epochs,
          steps_per_epoch=int(len(x_train)/128),
          verbose=1,
          validation_data=(x_test,y_test),
          shuffle=True,
          callbacks=[reduce_lr]
          )

Epoch 1/300
390/390 [==============================] - 4s 10ms/step - loss: 1.7692 - accuracy: 0.3375 - val_loss: 1.4342 - val_accuracy: 0.4791
Epoch 2/300
390/390 [==============================] - 4s 10ms/step - loss: 1.3843 - accuracy: 0.4979 - val_loss: 1.1765 - val_accuracy: 0.5730
Epoch 3/300
390/390 [==============================] - 4s 9ms/step - loss: 1.2136 - accuracy: 0.5644 - val_loss: 1.0641 - val_accuracy: 0.6230
Epoch 4/300
390/390 [==============================] - 4s 10ms/step - loss: 1.1071 - accuracy: 0.6055 - val_loss: 0.9808 - val_accuracy: 0.6492
Epoch 5/300
390/390 [==============================] - 4s 9ms/step - loss: 1.0380 - accuracy: 0.6318 - val_loss: 0.9268 - val_accuracy: 0.6707
Epoch 6/300
390/390 [==============================] - 4s 9ms/step - loss: 0.9847 - accuracy: 0.6519 - val_loss: 0.8796 - val_accuracy: 0.6917
Epoch 7/300
390/390 [==============================] - 4s 9ms/step - loss: 0.9364 - accuracy: 0.6677 - val_loss: 0.8571 - val_accuracy: 0.6

In [33]:
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5528826117515564
Test accuracy: 0.8108999729156494
